<a href="https://colab.research.google.com/github/cawcawlabs/ouster_lidar_appnotes/blob/main/SLAM_to_Minecraft_in_5_min.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Transform a lidar recording to a minecraft map


![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/mine_vs_amulet.png
)

Let's make a 3D map out of Ouster lidar (Top Left), and turn it into a Minecraft map feature (Top Right)!

![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/single-cave-scan.png)

This image is a visualization of a single lidar scan, out of a real mineshaft pcap recording.<br>

What if you want a coherent map out of entire sequential scans of the mine? 
The 3D points will have to be aggregated, and motion corrected. This is where a SLAM algorithm comes to rescue! <br>

There are many flavors of SLAM, but here, we use a WebSLAM, a convenient web tool from Ouster to convert .pcap to a global coordinate map in .ply form.

Based on LOAM, the WebSLAM finds optimal pose and map representation that best fits the given sequence of lidar + IMU observations (from the 6-DOF IMU within the sensor).  




##Generating SLAM output (already done for the example. Can skip this step for demo)

Upload your .pcap & json file on https://webslam.ouster.dev/

![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/webslam.png)

The SLAM process takes sometime, but .ply is what you need. 

![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/after_webslam.png)

Now, we grab and run with the .ply!

Note that .ply can come from other SLAM solutions. See our SLAM partners such as Kudan for more info.

After loading the .ply and visualising via open3d, you can see the motion corrected map, ready to be transformed into a minecraft feature! <br>

Figure: .ply visualized in open3d (warning, RAM usage heavy)

![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/FULL_MINE.png)


##Transform SLAM output to Mincraft world

1. Download Webslam output as output.ply <br>
2. Download Pre-generated superflat world "FLAT_WORLD" (creative, super flat, java 1.8.1)<br>
3. Install open3d and amulet-core


In [ ]:
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/short_mine.ply

In [ ]:
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/FLAT_WORLD.zip
!unzip /content/FLAT_WORLD.zip

In [ ]:
!pip install open3d #tested on 0.15.2

In [ ]:
!pip install amulet-core #tested on amulet-core-1.7.0 amulet-nbt-1.0.4 pymctranslate-1.1.0


4. Load .ply to open3d instance, use Amulet-core map editor to map the list of 3D coordinates as "Stone" minecraft block after scaling. Save the world map


In [ ]:
import numpy as np
import open3d as o3d

import amulet
from amulet.api.block import Block

from tqdm.auto import tqdm

voxel_size = 1
#increase/decrease for low/high resolution but watch memory usage

print("Step 1: (.pcap+json) -> WEBSLAM -> .ply is already done")

print("Step 2: .ply -> o3d pointcloud")

pcd = o3d.io.read_point_cloud('/content/short_mine.ply')

block_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=voxel_size)

block_coordinates = np.array(list(v.grid_index[[0, 2, 1]] for v in tqdm(block_grid.get_voxels())))

print("Step 3: Places Blocks in Minecraft")
game_version = ("java", (1, 18, 1))
# load the pre-generated super flat level "FLAT_WORLD"
level = amulet.load_level("/content/FLAT_WORLD")

position = (0,0,0) #(x,y,z)

#There are more efficient ways to place blocks. See Amulet's API example.
for pos in tqdm(block_coordinates):
  block = Block("minecraft", "stone")
  level.set_version_block((int)(pos[0])+position[0],(int)(pos[1])+position[1],(int)(pos[2])+position[2],
            "minecraft:overworld",
            game_version,
            block,
  )
        
level.save()
level.close()

print(".ply transported to FLAT_WORLD")

In [ ]:
!zip -r /content/OUTPUT.zip /content/FLAT_WORLD

5. Download the world map, and open in your own Mincraft (Java 1.8.1).

In [ ]:
from google.colab import files

files.download('/content/OUTPUT.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Extract OUTPUT.zip, grab "FLAT_WORLD" folder (now with our lidar point cloud), and open in Java 1.8.1 Minecraft.<br>
Your minecraft map folders typically live in /Home/.minecraft/saves <br>


![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/minecraft_java.png)

Look up!!

![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/minecraft_view1.png)

You can also use amulet_map_editor tool to inspect the ptcloud

https://github.com/Amulet-Team/Amulet-Map-Editor


![picture](https://storage.googleapis.com/data.ouster.io/concept-engineering/colab_images/minecraft_view2.png)

##DISCUSSION

1. .ply to digital asset generation is a pretty memory intensive operation! If you haven't noticed, we gave you an abridged version of the mineshaft slam output. If we gave the full version in ply, it would have terminated the google colab session. So we truncated the recording in pcap, ran a smaller data into the WebSLAM. The pixel sampling resolution in open3d also weighs heavily in the ability to render without exploding memory.
2. SLAM is not a guarantee of perfect map generation and its performance depends on the environment it's asked to process.  Environments like Tunnels and caves are challenging due to lack of unique features to grasp and resolve observation residuals with. You'll notice that the abridged slam output is a bit more warped and blurry compared to the initial picture of the full mineslam. Here, less data (short recording of a straight segment) hurt our WebSLAM performance.
3. The potential does not end with Minecraft! This is just a taste of what's possible in creating your digital twin with Ouster lidars!